## Library

In [114]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import time
import json
from konlpy.tag import Okt
import re
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [101]:
#HYPER PARAMETER
DATA_PATH = './data.json'

In [102]:
# data loading
def loadData():
    with open(DATA_PATH, "r") as f:
        data = json.load(f)
    title = []
    content = []
    for i in data:
        title.append(i["title"])
        content.append(i["content"])
    return title, content

In [104]:
title, content = loadData()

In [107]:
# tokenizer, tagging, lemmatizer, textCategorizer

  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18911 sha256=98f46de74834556c47074a9d2150801b8a8b2d2202b4b7f60b1480f4f01994af
  Stored in directory: /Users/khyo/Library/Caches/pip/wheels/78/c1/84/b83a2fd6f1d63e136cba74bac4126bee3b8705eef6486635fd
Successfully built keras-self-attention


In [115]:
Okt = Okt()

In [126]:
for i in content:
    print(Okt.pos(i[0]))

[('이번', 'Noun'), ('조사', 'Noun'), ('는', 'Josa'), ('지난해', 'Noun'), ('10월', 'Number'), ('15일', 'Number'), ('~', 'Punctuation'), ('24일', 'Number'), ('까지', 'Foreign'), ('서비스', 'Noun'), ('센터', 'Noun'), ('업체', 'Noun'), ('를', 'Josa'), ('이용', 'Noun'), ('한', 'Josa'), ('성인', 'Noun'), ('1268', 'Number'), ('명', 'Noun'), ('을', 'Josa'), ('대상', 'Noun'), ('으로', 'Josa'), ('온라인', 'Noun'), ('설문', 'Noun'), ('방식', 'Noun'), ('으로', 'Josa'), ('진행', 'Noun'), ('됐다', 'Verb'), ('.', 'Punctuation')]
[('회', 'Noun'), ('동이', 'Noun'), ('끝난', 'Verb'), ('후', 'Noun'), ('야당', 'Noun'), ('원내대표', 'Noun'), ('들', 'Suffix'), ('은', 'Josa'), ('김태우', 'Noun'), ('특검법', 'Noun'), ('도입', 'Noun'), ('과', 'Josa'), ('신재민', 'Noun'), ('전', 'Noun'), ('사무관', 'Noun'), ('의', 'Josa'), ('폭로', 'Noun'), ('를', 'Josa'), ('검증', 'Noun'), ('하기', 'Verb'), ('위', 'Noun'), ('한', 'Josa'), ('기획', 'Noun'), ('재정', 'Noun'), ('위원회', 'Noun'), ('소집', 'Noun'), ('·', 'Punctuation'), ('청문회', 'Noun'), ('등', 'Noun'), ('을', 'Josa'), ('더불어', 'Verb'), ('민주당', 'Noun'), ('에', 

[('나경원', 'Noun'), ('한국', 'Noun'), ('당', 'Suffix'), ('원내대표', 'Noun'), ('는', 'Josa'), ('이', 'Determiner'), ('날', 'Noun'), ('조', 'Noun'), ('장관', 'Noun'), ('임명', 'Noun'), ('이', 'Josa'), ('발표', 'Noun'), ('된', 'Verb'), ('후', 'Noun'), ('"', 'Punctuation'), ('참담하다', 'Adjective'), ('"', 'Punctuation'), ('며', 'Noun'), ('"', 'Punctuation'), ('결국', 'Adverb'), ('이', 'Noun'), ('정권', 'Noun'), ('은', 'Josa'), ('민심', 'Noun'), ('을', 'Josa'), ('거', 'Noun'), ('스르', 'Noun'), ('고', 'Josa'), ('개혁', 'Noun'), ('에', 'Josa'), ('반대', 'Noun'), ('하며', 'Verb'), ('공정', 'Noun'), ('과', 'Josa'), ('정의', 'Noun'), ('를', 'Josa'), ('내', 'Determiner'), ('팽', 'Noun'), ('개', 'Suffix'), ('치는', 'Verb'), ('그런', 'Adjective'), ('결정', 'Noun'), ('을', 'Josa'), ('했다', 'Verb'), ('.', 'Punctuation'), ('대한민국', 'Noun'), ('헌', 'Modifier'), ('정사', 'Noun'), ('상', 'Suffix'), ('가장', 'Noun'), ('불행한', 'Adjective'), ('사태', 'Noun'), ('로', 'Josa'), ('기록', 'Noun'), ('될', 'Verb'), ('것', 'Noun'), ('"', 'Punctuation'), ('이라고', 'Josa'), ('평가', 'Noun'), ('했

[('지난', 'Noun'), ('4일', 'Number'), ('발매', 'Noun'), ('된', 'Verb'), ('우주소녀', 'Noun'), ('의', 'Josa'), ('스페셜', 'Noun'), ('앨범', 'Noun'), ("'", 'Punctuation'), ('For', 'Alpha'), ('the', 'Alpha'), ('Summer', 'Alpha'), ("'(", 'Punctuation'), ('포', 'Noun'), ('더', 'Noun'), ('썸머', 'Noun'), (')', 'Punctuation'), ('는', 'Verb'), ('국내', 'Noun'), ('최대', 'Noun'), ('음반', 'Noun'), ('차트', 'Noun'), ('인', 'Josa'), ('한', 'Determiner'), ('터', 'Noun'), ('차트', 'Noun'), ('에서', 'Josa'), ('주간', 'Noun'), ('음반', 'Noun'), ('차트', 'Noun'), ('(', 'Punctuation'), ('2019.6', 'Number'), ('.', 'Punctuation'), ('3~2019', 'Number'), ('.', 'Punctuation'), ('6.9', 'Number'), ('기준', 'Noun'), (')', 'Punctuation'), ('1', 'Number'), ('위', 'Noun'), ('를', 'Josa'), ('차지', 'Noun'), ('하며', 'Verb'), ('높은', 'Adjective'), ('인기', 'Noun'), ('를', 'Josa'), ('증명', 'Noun'), ('했다', 'Verb'), ('.', 'Punctuation')]


IndexError: list index out of range

In [88]:
sess = tf.Session()
model = seq2seq(multi=True, hidden_size=hidden_size, num_layers=num_layers,
                    learning_rate=learning_rate, batch_size=batch_size,
                    vocab_size=vocab_size,
                    encoder_size=encoder_size, decoder_size=32,
                    forward_only=forward_only)
sess.run(tf.global_variables_initializer())
step_time, loss = 0.0, 0.0
current_step = 0
start = 0
end = batch_size
while current_step < 10000001:

    if end > len(title):
        start = 0
        end = batch_size

    # Get a batch and make a step
    start_time = time.time()
    encoder_inputs, decoder_inputs, targets, target_weights = tool.make_batch(encoderinputs[start:end],
                                                                              decoderinputs[start:end],
                                                                              targets_[start:end],
                                                                              targetweights[start:end])

    if current_step % steps_per_checkpoint == 0:
        for i in range(decoder_size - 2):
            decoder_inputs[i + 1] = np.array([word_to_ix['<PAD>']] * batch_size)
        output_logits = model.step(sess, encoder_inputs, decoder_inputs, targets, target_weights, True)
        predict = [np.argmax(logit, axis=1)[0] for logit in output_logits]
        predict = ' '.join(ix_to_word[ix][0] for ix in predict)
        real = [word[0] for word in targets]
        real = ' '.join(ix_to_word[ix][0] for ix in real)
        print('\n----\n step : %s \n time : %s \n LOSS : %s \n 예측 : %s \n 손질한 정답 : %s \n 정답 : %s \n----' %
              (current_step, step_time, loss, predict, real, title[start]))
        loss, step_time = 0.0, 0.0

    step_loss = model.step(sess, encoder_inputs, decoder_inputs, targets, target_weights, False)
    step_time += time.time() - start_time / steps_per_checkpoint
    loss += np.mean(step_loss) / steps_per_checkpoint
    current_step += 1
    start += batch_size
    end += batch_size

/Users/khyo/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:535: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.GRUCell` is deprecated and will be removed "


AttributeError: module 'tensorflow.compat.v1.nn' has no attribute 'seq2seq'

In [70]:
def make_dict_all_cut(contents, minlength, maxlength, jamo_delete=False):
    dict = defaultdict(lambda: [])
    for doc in contents:
        for i in range(len(doc)):
            for idx, word in enumerate(doc[i].split()):
                if len(word) > minlength:
                    normalizedword = word[:maxlength]
                    if jamo_delete:
                        tmp = []
                        for char in normalizedword:
                            if ord(char) < 12593 or ord(char) > 12643:
                                tmp.append(char)
                        normalizedword = ''.join(char for char in tmp)
                    if word not in dict[normalizedword]:
                        dict[normalizedword].append(word)
    print(dict)
    dict = sorted(dict.items(), key=itemgetter(0))[1:]
    words = []
    for i in range(len(dict)):
        word = []
        word.append(dict[i][0])
        for w in dict[i][1]:
            if w not in word:
                word.append(w)
        words.append(word)

    words.append(['<PAD>'])
    words.append(['<S>'])
    words.append(['<E>'])
    words.append(['<UNK>'])
    # word_to_ix, ix_to_word 생성
    ix_to_word = {i: ch[0] for i, ch in enumerate(words)}
    word_to_ix = {}
    for idx, words in enumerate(words):
        for word in words:
            word_to_ix[word] = idx
    print('컨텐츠 갯수 : %s, 단어 갯수 : %s'
                  % (len(contents), len(ix_to_word)))
    return word_to_ix, ix_to_word

In [79]:
word_to_ix, ix_to_word = make_dict_all_cut(content,2, 4)

defaultdict(<function make_dict_all_cut.<locals>.<lambda> at 0x7fb815697430>, {'조사는': ['조사는'], '지난해': ['지난해'], '10월': ['10월'], '15일~': ['15일~24일까지'], '서비스센': ['서비스센터', '서비스센터가', '서비스센터의', '서비스센터는'], '업체를': ['업체를'], '이용한': ['이용한'], '1268': ['1268명을'], '대상으로': ['대상으로', '대상으로는'], '온라인설': ['온라인설문'], '방식으로': ['방식으로'], '진행됐다': ['진행됐다.'], '서비스': ['서비스'], '품질·상': ['품질·상품·호감도'], '부문을': ['부문을'], '총체적으': ['총체적으로'], '평가한': ['평가한'], '종합만족': ['종합만족도는', '종합만족도'], '국산차': ['국산차'], '3.60': ['3.60점,'], '수입차': ['수입차'], '3.54': ['3.54점이었다.'], '결과를': ['결과를'], '살펴보면': ['살펴보면'], '서비스의': ['서비스의'], '일관성,': ['일관성,'], '전문성,': ['전문성,'], "'서비스": ["'서비스"], "품질'에": ["품질'에"], '만족도는': ['만족도는'], '3.77': ['3.77점이었다.'], '3.72': ['3.72점으로,'], '렉서스': ['렉서스'], '3.93': ['3.93점,'], '메르세데': ['메르세데스'], '3.86': ['3.86점,'], '3.80': ['3.80점'], '순이었다': ['순이었다.'], "만족도'": ["만족도'는"], '편리성,': ['편리성,'], '점검절차': ['점검절차'], '비용의': ['비용의'], '적절성,': ['적절성,'], '사이트·': ['사이트·앱'], '정보유용': ['정보유용성'], '요인으로': ['요인으로'], '평가했는': ['평가했는데,'], '국산·수':

In [44]:
for doc in content:
    print(doc[0])

이번 조사는 지난해 10월 15일~24일까지 서비스센터 업체를 이용한 성인 1268명을 대상으로 온라인설문 방식으로 진행됐다.
회동이 끝난 후 야당 원내대표들은 김태우 특검법 도입과 신재민 전 사무관의 폭로를 검증하기 위한 기획재정위원회 소집·청문회 등을 더불어민주당에 요청했지만, 홍영표 민주당 원내대표(인천 부평을)가 수용 불가 입장을 밝혔다고 전했다.
수천억 원을 들여 조성하게 될 인천뮤지엄파크가 자칫 속 빈 강정이 될 수 있다는 우려가 나오고 있다.
주요 변경 내용은 가산금리 인하(은행별 상이)와 상환방식 변경, 금융기관(국민은행) 변경 등이다.
각종 논란들이 불식되지 않은 상황에서 조합 내 갈등이 깊어지면서 일부 조합원 측은 비대위 결성 조짐을 내비치고 있는 것으로 알려졌다.
28일 제작사 과수원뮤지컬컴퍼니가 뮤지컬 '루드윅 : 베토벤 더 피아노'에서 청년 역을 맡은 배우 이용규, 강찬, 박준휘, 조환지의 여심을 사로잡는 캐릭터 컨셉컷을 공개해 이목을 집중시켰다.
시에서는 19일, 김정섭 시장 명의로 공주보 기능을 유지해줄 것을 요구하는 건의문을 직접 작성해 국무총리, 환경부장관, 청와대 정책실장, 더불어민주당 대표 등에게 전달해 공주시민들의 뜻을 존중해줄 것을 건의했다.
비산먼지 사업장 203개소 및 대기배출사업장 178곳에 대한 지도·점검을 강화하고 환경정책실명제를 전면 실시한다.
보강방법으로 외장재 교체, 스프링클러 및 외부피난계단 설치 등을 선택할 수 있다.
28일 시와 상인회 등에 따르면 이마트가 노브랜드는 지난달 22일 군포시에 준(準)대규모점포 개설계획 예고와 등록신청을 낸뒤 군포시 번영로의 한 상가건물에 211.245㎡규모의 매장을 갖추고 지난 23일 출점 오픈했다.
서울반도체는 "방사선과 방사능은 다르며 따라서 장비 주변에 잔류방사선이 존재하지 않으며, 방사능 물질 역시 공장 어느 곳에도 전혀 없다"면서 "이번 사고는 X-ray 측정기에 안전장치인 스위치를 테이프로 붙이고 손을 안쪽으로 넣고 반복 검사하여 발생하였다"고 설명했다.
이번 

IndexError: list index out of range

In [53]:
content[0]

['이번 조사는 지난해 10월 15일~24일까지 서비스센터 업체를 이용한 성인 1268명을 대상으로 온라인설문 방식으로 진행됐다.',
 '서비스 품질·상품·호감도 3개 부문을 총체적으로 평가한 종합만족도는 국산차 서비스센터가 평균 3.60점, 수입차 서비스센터가 평균 3.54점이었다.',
 "세부 항목 평가 결과를 살펴보면 서비스센터 서비스의 일관성, 전문성, 고객 대응 등 '서비스 품질'에 대한 만족도는 국산차 서비스센터의 경우 평균 3.77점이었다.",
 '수입차 서비스센터는 평균 3.72점으로, 렉서스 3.93점, 메르세데스 벤츠 3.86점, 포드 3.80점 등의 순이었다.',
 "'서비스 상품 만족도'는 시설 이용 편리성, 점검절차 및 비용의 적절성, 사이트·앱 정보유용성 3개 요인으로 평가했는데, 국산·수입차 서비스센터 모두 시설 이용 편리성 만족도가 각각 3.71점과 3.88점으로 높은 반면, 사이트·앱 정보유용성 만족도는 3.29점으로 낮았다.",
 '국산차 서비스센터의 경우 르노삼성이 모든 요인에서 만족도가 상대적으로 높았고, 수입차 서비스센터는 렉서스가 점검절차 및 비용의 적절성에서, 벤츠가 시설 이용 편리성에서 만족도가 상대적으로 높았다.',
 "소비자가 서비스를 체험하면서 느낀 주관적 감정을 평가한 '서비스 호감도'는 국산차 서비스센터의 경우 르노삼성 3.69점, 쌍용자동차 3.62점 등의 순이었다.",
 '한편 피해구제 합의율은 BMW와 벤츠가 상대적으로 높았다.',
 '지난 2015년부터 최근 3년간 소비자원에 접수된 11개 자동차 서비스센터 관련 피해구제 신청건수는 총 916건으로 집계됐다.',
 '등록대수 10만 대당 건수는 현대자동차가 1.4건으로 가장 적었고 다음으로 렉서스, 기아자동차 등의 순이었다.',
 '피해구제 접수사건 중 환급, 배상, 계약해제 등 보상 처리된 사건의 비율(합의율)은 BMW와 벤츠가 각각 66.7%, 57.8%인데 비해 아우디(38.2%), 쌍용자동차(49.0%)의 합의율은 상대적으로 낮게 나타났다.',


In [45]:
cont

NameError: name 'cont' is not defined

In [37]:
na = np.array(content)

/var/folders/33/2xrkp0fs5vb89z_lhx32tp480000gn/T/ipykernel_45662/1967311761.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  na = np.array(content)


In [41]:
na[1][0]

'회동이 끝난 후 야당 원내대표들은 김태우 특검법 도입과 신재민 전 사무관의 폭로를 검증하기 위한 기획재정위원회 소집·청문회 등을 더불어민주당에 요청했지만, 홍영표 민주당 원내대표(인천 부평을)가 수용 불가 입장을 밝혔다고 전했다.'

In [73]:
a

{'"\'하치': 0,
 '"\'하치이야기\'는': 0,
 '"(노래': 1,
 '"(노래)': 1,
 '"(이번': 2,
 '"11공': 3,
 '"11공구를': 3,
 '"1년': 4,
 '"201': 5,
 '"2016년': 5,
 '"2019년': 5,
 '"2017년도부터': 5,
 '"2018년': 5,
 '"2분기': 6,
 '"5월': 7,
 '"8억': 8,
 '"AKM': 9,
 '"AKMU의': 9,
 '"AMI': 10,
 '"DLS': 11,
 '"IFE': 12,
 '"IFEZ가': 12,
 '"XX': 13,
 '"가정간': 14,
 '"가정간편식': 14,
 '"가천대': 15,
 '"가천대의': 15,
 '"가해': 16,
 '"감독님': 17,
 '"감독님께서': 17,
 '"감사원': 18,
 '"감사원의': 18,
 '"개발행': 19,
 '"개발행위제한으로': 19,
 '"결국': 20,
 '"경산시': 21,
 '"경산시가': 21,
 '"경제': 22,
 '"경제자': 23,
 '"경제자유구역을': 23,
 '"경찰과': 24,
 '"계란뿐': 25,
 '"계란뿐만': 25,
 '"계란에': 26,
 '"계속되': 27,
 '"계속되는': 27,
 '"공공주': 28,
 '"공공주택지구': 28,
 '"공동체': 29,
 '"공동체가': 29,
 '"공연장': 30,
 '"공연장치': 30,
 '"관련': 31,
 '"구체적': 32,
 '"구체적으로': 32,
 '"국내': 33,
 '"국민': 34,
 '"국민들': 35,
 '"국민들의': 35,
 '"국비확': 36,
 '"국비확보의': 36,
 '"국제적': 37,
 '"국회선': 38,
 '"국회선진화법': 38,
 '"국회에': 39,
 '"국회에서': 39,
 '"그간': 40,
 '"그동안': 41,
 '"그동안의': 41,
 '"그래': 42,
 '"그러나': 43,
 '"그런': 44,
 '"그럼에': 45,
 '"그럼에도': 45,
 '"금강의': 

In [74]:
b

{0: '"\'하치',
 1: '"(노래',
 2: '"(이번',
 3: '"11공',
 4: '"1년',
 5: '"201',
 6: '"2분기',
 7: '"5월',
 8: '"8억',
 9: '"AKM',
 10: '"AMI',
 11: '"DLS',
 12: '"IFE',
 13: '"XX',
 14: '"가정간',
 15: '"가천대',
 16: '"가해',
 17: '"감독님',
 18: '"감사원',
 19: '"개발행',
 20: '"결국',
 21: '"경산시',
 22: '"경제',
 23: '"경제자',
 24: '"경찰과',
 25: '"계란뿐',
 26: '"계란에',
 27: '"계속되',
 28: '"공공주',
 29: '"공동체',
 30: '"공연장',
 31: '"관련',
 32: '"구체적',
 33: '"국내',
 34: '"국민',
 35: '"국민들',
 36: '"국비확',
 37: '"국제적',
 38: '"국회선',
 39: '"국회에',
 40: '"그간',
 41: '"그동안',
 42: '"그래',
 43: '"그러나',
 44: '"그런',
 45: '"그럼에',
 46: '"금강의',
 47: '"금융상',
 48: '"금융소',
 49: '"금융시',
 50: '"금융회',
 51: '"기본권',
 52: '"기존',
 53: '"기존의',
 54: '"긴급한',
 55: '"김태우',
 56: '"꿈과',
 57: '"내포신',
 58: '"낸드플',
 59: '"노브랜',
 60: '"노재헌',
 61: '"노태우',
 62: '"다른',
 63: '"다양한',
 64: '"대기업',
 65: '"대덕e',
 66: '"대덕구',
 67: '"대덕의',
 68: '"대두',
 69: '"대박이',
 70: '"대임지',
 71: '"대표적',
 72: '"더욱',
 73: '"도가',
 74: '"도는',
 75: '"도시의',
 76: '"도시철',
 77: '"도쿄에',
 78: '"때문에',
 7

In [113]:
content[1]

['회동이 끝난 후 야당 원내대표들은 김태우 특검법 도입과 신재민 전 사무관의 폭로를 검증하기 위한 기획재정위원회 소집·청문회 등을 더불어민주당에 요청했지만, 홍영표 민주당 원내대표(인천 부평을)가 수용 불가 입장을 밝혔다고 전했다.',
 '나경원 한국당 원내대표는 "오늘 가장 강하게 요청한 것은 특검법이었다"고 말했다.',
 '나 원내대표는 "김태우 수사관이 제기한 의혹은 운영위에서 차고 넘치는 증거를 발견할 수 있었다"라며 "이와 관련된 상임위 소집을 요구하고 있는데 실질적으로 여당이 전혀 협조하고 있지 않다"라고 지적했다.',
 '이어 "신재민 전 사무관이 제기한 의혹에 대해서도 기재위가 실질적으로 김동연 전 경제부총리를 부를 수 있어야 할 것"이라며 "지난번 운영위 진행에서 느낄 수 있었지만 일방적 상임위에 한계를 느꼈기 때문에 청문회를 진행하는 것에 대해서도 논의하고 싶다"고 덧붙였다.',
 '김관영 바른미래당 원내대표도 "신재민 전 사무관 건은 기재위 차원의 청문회를 일관되게 저희 당이 주장해왔다"라며 "전 국회가 국민적 의혹이 있는 사안에 대해 신속하게 청문회 내지는 위원회 열어서 그런 점들에 대해 의혹을 규명하는 게 필요하다고 생각한다"라고 밝혔다.',
 '이어 "국회에서 현안 관련 상임위 개최 청문회, 소규모 단위 국정조사는 자주 개최돼야 한다"고 강조했다.',
 '김 원내대표는 "국회선진화법 개정, 법사위 개선 등 문제들을 운영위 제도 개선 소위에서 신속히 논의해서 개선이 돼야 할 것"이라며 "선거법 제도 개혁, 법안소위 주 1회 소집, 특별감찰관 임명문제, 채용비리 국정조사 계획서 문제 등이 2월 국회에서 꼭 결실 맺을 수 있게 해야 할 것"이라고도 했다.',
 '반면 홍영표 민주당 원내대표는 말을 아꼈다.',
 '홍 원내대표는 모두발언을 통해 "새해를 맞아서 국회가 대화와 타협으로 민생을 챙기고 한반도 평화를 준비하는 국회가 될 수 있게 최선을 다하겠다"고 했다.',
 '한편 신재민 전 기획재정부 사무관을 원색적으로 비난한 손혜원 민주당 

In [112]:
title[1]

'야당 ‘김태우·신재민 협공‘, 여당 "수용불가"'

In [ ]:
pred[1]